# 1. RNN从零实现

In [1]:
%matplotlib
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


Using matplotlib backend: <object object at 0x7f6f68c0f2e0>


In [2]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 1. 独热编码-one hot
简⾔之，将每个索引映射为相互不同的单位向量：假设词表中不同词元的数⽬为N（即len(vocab)），词元索引的范围为0到N − 1。如果词元的索引是整数i，那么我们将创建⼀个⻓度为N的全0向量，并将第i处的元素设置为1。此向量是原始词元的⼀个独热向量。

In [3]:
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

我们每次采样的⼩批量数据形状是⼆维张量：（批量⼤⼩，时间步数）。one_hot函数将这样⼀个⼩批量数据转
换成三维张量，张量的最后⼀个维度等于词表⼤⼩（len(vocab)）。我们经常转换输⼊的维度，以便获得形状
为（时间步数，批量⼤⼩，词表⼤⼩）的输出。这将使我们能够更⽅便地通过最外层的维度，⼀步⼀步地更
新⼩批量数据的隐状态。


In [4]:
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape

torch.Size([5, 2, 28])

## 2. 初始化模型参数
接下来，我们初始化循环神经⽹络模型的模型参数。隐藏单元数num_hiddens是⼀个可调的超参数。当训练语⾔模型时，输⼊和输出来⾃相同的词表。因此，它们具有相同的维度，即词表的⼤⼩。


In [5]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size
    
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01
    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params


## 3. RNN模型
为了定义循环神经⽹络模型，我们⾸先需要⼀个init_rnn_state函数在初始化时返回隐状态。这个函数的返回是⼀个张量，张量全⽤0填充，形状为（批量⼤⼩，隐藏单元数）。

In [6]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

下⾯的rnn函数定义了如何在⼀个时间步内计算隐状态和输出。循环神经⽹络模型通过inputs最外层的维度实现循环，以便逐时间步更新⼩批量数据的隐状态H。此外，这⾥使⽤tanh函数作为激活函数。如 之前所述，当元素在实数上满⾜均匀分布时，tanh函数的平均值为0。


In [7]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量⼤⼩，词表⼤⼩)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量⼤⼩，词表⼤⼩)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)


In [8]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经⽹络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
        get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn
    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)


In [9]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                        init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

## 4. 预测
让我们⾸先定义预测函数来⽣成prefix之后的新字符，其中的prefix是⼀个⽤⼾提供的包含多个字符的字符
串。在循环遍历prefix中的开始字符时，我们不断地将隐状态传递到下⼀个时间步，但是不⽣成任何输出。这
被称为预热（warm-up）期，因为在此期间模型会⾃我更新（例如，更新隐状态），但不会进⾏预测。预热期
结束后，隐状态的值通常⽐刚开始的初始值更适合预测，从⽽预测字符并输出它们。

In [10]:
def predict_ch8(prefix, num_preds, net, vocab, device): #@save
    """在prefix后⾯⽣成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]: # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds): # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [11]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

'time traveller irbwndsngq'

鉴于我们还没有训练⽹络，它会⽣成荒谬的预测结果。


## 5. 梯度裁剪
对于⻓度为T的序列，我们在迭代中计算这T个时间步上的梯度，将会在反向传播过程中产⽣⻓度为O(T)的矩阵乘法链。如之前所述，当T较⼤时，它可能导致数值不稳定，例如可能导致梯度爆炸或梯度消失。因此，循环神经⽹络模型往往需要额外的⽅式来⽀持稳定训练。

In [12]:
def grad_clipping(net, theta): #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

## 6. 训练
在训练模型之前，让我们定义⼀个函数在⼀个迭代周期内训练模型。它与我们之前训练模型的⽅式有三个不同之处。
+ 1. 序列数据的不同采样⽅法（随机采样和顺序分区）将导致隐状态初始化的差异。
+ 2. 我们在更新模型参数之前裁剪梯度。这样的操作的⽬的是，即使训练过程中某个点上发⽣了梯度爆炸，也能保证模型不会发散。
+ 3. 我们⽤困惑度来评价模型。如前面所述，这样的度量确保了不同⻓度的序列具有可⽐性。

具体来说，当使⽤顺序分区时，我们只在每个迭代周期的开始位置初始化隐状态。由于下⼀个⼩批量数据中的第i个⼦序列样本与当前第i个⼦序列样本相邻，因此当前⼩批量数据最后⼀个样本的隐状态，将⽤于初始化下⼀个⼩批量数据第⼀个样本的隐状态。这样，存储在隐状态中的序列的历史信息可以在⼀个迭代周期内流经相邻的⼦序列。然⽽，在任何⼀点隐状态的计算，都依赖于同⼀迭代周期中前⾯所有的⼩批量数据，这使得梯度计算变得复杂。为了降低计算量，在处理

In [13]:
#@save
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练⽹络⼀个迭代周期（定义⻅第8章）"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2) # 训练损失之和,词元数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第⼀次迭代或使⽤随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                # state对于nn.LSTM或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调⽤了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()


In [14]:
#@save
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
                use_random_iter=False):
    """训练模型（定义⻅第8章）"""
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
                        net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))


In [15]:
num_epochs, lr= 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

困惑度 1.0, 17752.1 词元/秒 cpu
time travelleryou can show black is white by argument said filby
traveller with a slight accession ofcheerfulness really thi


In [16]:
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                     init_rnn_state, rnn)
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(),
         use_random_iter=True)

困惑度 1.5, 21117.8 词元/秒 cpu
time traveller held in his hind why caunot engott rowe thing he 
travellerit s against reason said filbywan argementas thes 


## 总结
+ 我们可以训练⼀个基于循环神经⽹络的字符级语⾔模型，根据⽤⼾提供的⽂本的前缀⽣成后续⽂本。
+ ⼀个简单的循环神经⽹络语⾔模型包括输⼊编码、循环神经⽹络模型和输出⽣成。
+ 循环神经⽹络模型在训练以前需要初始化状态，不过随机抽样和顺序划分使⽤初始化⽅法不同。
+ 当使⽤顺序划分时，我们需要分离梯度以减少计算量。
+ 在进⾏任何预测之前，模型通过预热期进⾏⾃我更新（例如，获得⽐初始值更好的隐状态）。
+ 梯度裁剪可以防⽌梯度爆炸，但不能应对梯度消失。

# 2. 简介实现


In [17]:
import torch 
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 1. 定义模型
我们构造⼀个具有256个隐藏单元的单隐藏层的循环神经⽹络
层rnn_layer。

In [18]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

我们使⽤张量来初始化隐状态，它的形状是（隐藏层数，批量⼤⼩，隐藏单元数）。


In [19]:
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

torch.Size([1, 32, 256])

通过⼀个隐状态和⼀个输⼊，我们就可以⽤更新后的隐状态计算输出。需要强调的是，rnn_layer的“输出”（Y）
不涉及输出层的计算：它是指每个时间步的隐状态，这些隐状态可以⽤作后续输出层的输⼊。

In [20]:
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

In [24]:
#@save
class RNNModel(nn.Module):
    """循环神经⽹络模型"""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)
    
    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层⾸先将Y的形状改为(时间步数*批量⼤⼩,隐藏单元数)
        # 它的输出形状是(时间步数*批量⼤⼩,词表⼤⼩)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state
    
    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return torch.zeros((self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens),
                        device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                    self.num_directions * self.rnn.num_layers,
                    batch_size, self.num_hiddens), device=device),
                        torch.zeros((
                            self.num_directions * self.rnn.num_layers,
                            batch_size, self.num_hiddens), device=device))


## 2. 预测与训练

In [25]:
device = d2l.try_gpu()
net = RNNModel(rnn_layer, vocab_size=len(vocab))
net = net.to(device)
d2l.predict_ch8('time traveller', 10, net, vocab, device)

'time travellerswswswswsw'

In [26]:
num_epochs, lr = 500, 1
d2l.train_ch8(net, train_iter, vocab, lr, num_epochs, device)

perplexity 1.3, 32935.8 tokens/sec on cpu
time traveller for so it will spocinge that thes hould as ravely
traveller amot far abal from thepresent moment our mental e
